# Ensemble Modeling: Model Generation

In [1]:
import warnings
from operator import attrgetter
from os import path

from cobra import DictList
from cobra.sampling import sample

import numpy as np

from scipy import optimize

import roadrunner

import sympy as sym

import mass
from mass import MassConfiguration, UnitDefinition
from mass.enzyme_modules import EnzymeModule
from mass.io.json import load_json_model, save_json_model
from mass.io.sbml import write_sbml_model

from mass.simulation import (
    ensemble, generate_ensemble_of_models)
from mass.thermo import ConcSolver, sample_concentrations
from mass.util import Keq2k, k2Keq, strip_time

from table_export import export_csv_files_for_models

# Configure roadrunner to allow for more output rows
roadrunner.Config.setValue(
    roadrunner.Config.MAX_OUTPUT_ROWS, 1e6)

mass_config = MassConfiguration()
mass_config.decimal_precision = 12  # Round after 12 digits after decimal
mass_config.solver = "cplex"
print("MASSpy version: " + mass.__version__)

MASSpy version: 0.1.1


## Load Reference Models

In [2]:
glycolysis = load_json_model(
    path.abspath(path.join(
        "data", "reference_models", "JSON", "Glycolysis.json")))
reference_model = load_json_model(
    path.abspath(path.join(
        "data", "reference_models", "JSON", "Glycolysis_Hb.json")))
PYK =  load_json_model(
    path.abspath(path.join(
        "data", "reference_models", "JSON", "PYK.json")))

KeyboardInterrupt: 

## Generate Ensemble Data via Sampling

In [ ]:
f_samples = 15
flux_percent_deviation = 0.8

c_samples = 15
conc_percent_deviation = 0.8

seed = int(4)
print("Number of combinations: {}".format(f_samples * c_samples))

### Flux Sampling

In [ ]:
for reaction in glycolysis.reactions:
    reaction.bounds = sorted([
        round(reaction.steady_state_flux * (1 - flux_percent_deviation),
              mass_config.decimal_precision),
        round(reaction.steady_state_flux * (1 + flux_percent_deviation),
              mass_config.decimal_precision)])

flux_samples = sample(glycolysis, n=f_samples, seed=seed)

### Concentration Sampling

In [ ]:
conc_solver = ConcSolver(
    glycolysis,
    excluded_metabolites=["h_c", "h2o_c"],
    equilibrium_reactions=["ADK1"],
    constraint_buffer=1e-7)

conc_solver.setup_sampling_problem(
    conc_percent_deviation=conc_percent_deviation,
    Keq_percent_deviation=0)

conc_samples = sample_concentrations(conc_solver, n=c_samples, seed=seed)

## Create Ensemble
### Create Models from Data

In [ ]:
# Generate candidate model states
models = generate_ensemble_of_models(
    reference_model=reference_model,
    flux_data=flux_samples,
    conc_data=conc_samples)
print("# candidate models: {0}".format(len(models)))

In [ ]:
# Calculate PERCs for candidate models and 
# remove those that produce negative PERCS (thermdynamically infeasible)
models, infeasible = ensemble.ensure_positive_percs(
    models=models, reactions=[
        r.id for r in reference_model.reactions
        if r not in reference_model.boundary
        and r.id not in ["HBO1", "HBO2", "HBO3", "HBO4", "HBDPG"]],
    update_values=True)
print("# Infeasible, negative PERCs: {0}".format(len(infeasible)))

In [ ]:
# Ensure model can simulate to steady state
models, infeasible = ensemble.ensure_steady_state(
    models=models, strategy="simulate",
    update_values=True, decimal_precision=True,
    tfinal=1e5, solver_options={
        "absolute_tolerance": 1e-15, 
        "relative_tolerance": 1e-9})
print("# Infeasible, no steady state found: {0}".format(len(infeasible)))

In [ ]:
# Ensure model can simulate to steady state with ATP disturbance
models, infeasible = ensemble.ensure_steady_state(
    models=models, strategy="simulate",
    perturbations={"kf_ATPM": "kf_ATPM * 1.5"},
    update_values=False, decimal_precision=True,
    tfinal=1e5, solver_options={
        "absolute_tolerance": 1e-15, 
        "relative_tolerance": 1e-9})
print("# Infeasible, disturbance not rejected: {0}".format(len(infeasible)))

### Create PYK EnzymeModules for Models
#### Determine steady state concentrations symbolically

In [ ]:
# Get dict of ODEs for enzyme forms
ode_dict = {
    sym.Symbol(e_mod_form.id): sym.Eq(strip_time(e_mod_form.ode), 0)
    for e_mod_form in PYK.enzyme_module_forms}
# Get enzyme module forms
enzyme_module_forms = PYK.enzyme_module_forms.copy()
# Reverse list for increased performance (due to symmetry assumption)
# by solving for the most activated/inhibitors bound first.
enzyme_module_forms.reverse()

enzyme_solutions = {}
for enzyme_module_form in enzyme_module_forms:
    # Skip dependent variable
    if "pyk_R0_c" == str(enzyme_module_form):
        continue
    enzyme_module_form = sym.Symbol(enzyme_module_form.id)
    # Susbtitute in previous solutions, solve for the enzyme module form
    equation = ode_dict[enzyme_module_form]
    sol = sym.solveset(equation.subs(enzyme_solutions),
                       enzyme_module_form)
    enzyme_solutions[enzyme_module_form] = list(sol)[0]
    # Update the dictionary of solutions with the solutions
    enzyme_solutions.update({
        enzyme_module_form: sol.subs(enzyme_solutions) 
        for enzyme_module_form, sol in enzyme_solutions.items()})
    
# Solve for last unknown concentration symbolically
enzyme_rate_equation_error = strip_time(PYK.enzyme_rate_error())
sol = sym.solveset(enzyme_rate_equation_error.subs(enzyme_solutions),
                   "pyk_R0_c")
# Update solution dictionary with the new solution
enzyme_solutions[sym.Symbol("pyk_R0_c")] = list(sol)[0]

# Update solutions with free variable solutions
enzyme_solutions = {
    enzyme_module_form: sym.simplify(solution.subs(enzyme_solutions))
    for enzyme_module_form, solution in enzyme_solutions.items()}

#### Determine PYK rate parameters, steady state concentrations, and add to model

In [ ]:
def get_numerical_values(model):
    """Return a dict of numerical values to substutite into equations."""
    # Get equilibrium constants
    numerical_values = {
        PYK.reactions.PYK_L.Keq_str: PYK.reactions.PYK_L.Keq}
    numerical_values.update({
        param: value for param, value in PYK.custom_parameters.items()
        if param.startswith("Keq")})

    # Get initial conditions
    numerical_values.update({
        met.id: met.ic
        for met in model.metabolites.get_by_any(
            PYK.enzyme_module_ligands.list_attr("id"))})
    rxn = model.reactions.get_by_id("PYK")
    # Get steady state flux of enzyme
    numerical_values.update({
        PYK.enzyme_flux_symbol_str: rxn.steady_state_flux
    })
    return numerical_values

def get_total_constraint(enzyme_sols):
    """Return the total enzyme constraint in terms of rate constants."""
    # Get constraint
    enzyme_total_constraint = abs(strip_time(
        PYK.enzyme_concentration_total_error(use_values=False)))
    # Substitute values into constraint and simplify
    enzyme_total_constraint = enzyme_total_constraint.subs({
        PYK.enzyme_total_symbol_str: PYK.enzyme_concentration_total})
    enzyme_total_constraint = sym.simplify(
        enzyme_total_constraint.subs(enzyme_sols))
        
    return enzyme_total_constraint

def copy_module_and_add_numerical_values(model, numerical_values):
    """Copy the non-parameterized PYK enzyme and set numerical values."""
    PYK_new = PYK.copy()
    for met in PYK.metabolites:
        met.ic = numerical_values.pop(met.id)
    del numerical_values["v_PYK"]
    PYK_new.update_parameters(numerical_values)
    
    return PYK_new

def create_PYK_module(model, enzyme_sols=None):
    """Create PYK module using the given model for numerical values."""
    # Get numerical values from the model
    numerical_values = get_numerical_values(model)
    # Sub values into equations for enzyme forms
    enzyme_sols = {
        enzyme_module_form: sym.simplify(solution.subs(numerical_values))
        for enzyme_module_form, solution in enzyme_sols.items()}
    # Get arguments, should only be 3 ratge constants
    args = set()
    for sol in enzyme_sols.values():
        args.update(sol.atoms(sym.Symbol))
    assert len(args) == 3
    # Get enzyme total constraint and substitute values
    enzyme_total_constraint = get_total_constraint(enzyme_sols)

    # Create a sorted tuple of the arguments
    args = tuple(sorted([str(arg) for arg in list(args)]))
    # Create the objective function as a lambda function
    obj_func = lambda x: sym.lambdify(args, enzyme_total_constraint)(*x)

    # Find a feasible solution
    initial_guess = [6e3, 4e6, 4e6]
    (kf_lb, kf_ub) = (1e0, 1e9)
    kf_bounds = ((kf_lb, kf_ub), (kf_lb, kf_ub), (kf_lb, kf_ub))
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        sol = optimize.minimize(
            obj_func, x0=initial_guess, method="trust-constr",
            bounds=kf_bounds,
            options={"gtol": 1e-12, "xtol": 1e-12,
                     "maxiter": 1e4, "disp": False})
    # Could not find a solution, no module constructed
    if not sol.success:
        return None

    # Update the paramter values dict with the feasible solution
    numerical_values.update(
        dict(zip(args, [round(x) for x in sol.x])))

    # Add the activation, inhibition, and allosteric rate constants
    for abbrev, value in zip(["I", "ACT", "L"], [1e6, 1e6, 1e6**2]):
        # Account for the enzyme prefix
        to_join = ("kf", PYK.id, abbrev)
        param = "_".join(to_join)
        numerical_values.update({param: value})

    # Substitute values into equations
    numerical_values.update({
        str(e_form): float(sym.simplify(solution.subs(numerical_values)))
        for e_form, solution in enzyme_sols.items()})
    
    PYK_new = copy_module_and_add_numerical_values(
        model, numerical_values)
    
    return PYK_new

# Create modules and add to models
models_wo_PYK = []
models_w_PYK = []
for i, model in enumerate(models):
    if i % int(len(models) * 0.1) == 0:
        print("About {0} percent finished".format(
            int(i/int(len(models) * 0.1))*10))
    PYK_new = create_PYK_module(model, enzyme_sols=enzyme_solutions)
    if PYK_new is None:
        models_wo_PYK += [model]
    else:
        model = model.merge(PYK_new, inplace=False)
        model.remove_reactions([model.reactions.get_by_id("PYK")])
        models_w_PYK += [model]
        model.id = "_".join((
            reference_model.id, PYK_new.id, 
            model.id[len(reference_model.id) + 1:-4]))

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    print("Running simulations for SS")
    found_ss, no_ss = ensemble.ensure_steady_state(
        models_w_PYK, strategy="simulate",
        update_values=True,
        tfinal=1e5, solver_options={
            "absolute_tolerance": 1e-15, 
            "relative_tolerance": 1e-9})
    print("Running simulations for SS w/ perturbation")
    ensemble_of_models, no_ss_w_pert = ensemble.ensure_steady_state(
        found_ss, strategy="simulate",
        perturbations={"kf_ATPM": "kf_ATPM * 1.5"},
        update_values=False,
        tfinal=1e5, solver_options={
            "absolute_tolerance": 1e-15, 
            "relative_tolerance": 1e-9})

print("Models with PYK, found SS w/ perturbation: {0}".format(
    len(ensemble_of_models)))
print("Models with PYK, no SS w/ perturbation: {0}".format(
    len(no_ss_w_pert)))
print("Models with PYK, no SS: {0}".format(
    len(no_ss)))
print("Models without PYK: {0}".format(
    len(models_wo_PYK)))

## Export Models

In [ ]:
for model in ensemble_of_models:
    # Export model as JSON
    save_json_model(
        mass_model=model, 
        filename=path.abspath(
            path.join("data", "ensemble_models", "JSON", model.id  + ".json")))
    # Export model as SBML
    write_sbml_model(
        mass_model=model,
        filename=path.abspath(
            path.join("data", "ensemble_models", "SBML", model.id  + ".xml.zip")))
# Export value tables
export_csv_files_for_models(ensemble_of_models, "for_ensemble_generation_notebook")